In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import optuna
import lightgbm
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score

import os
import yaml
import sys

sys.path.append('../src')
import utils.utils as utils

!pip freeze > ../requirements.txt

c:\Users\HNg\Desktop\Researches\202507_homecredit_pd\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('../data/cleaned_data.csv')

In [4]:
X = data.drop(['TARGET', 'SK_ID_CURR'], axis=1)
y = data['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=810)

# Double check categorical cols
cat_cols = [col for col in X.columns if X[col].dtype == 'object' or str(X[col].dtype) == 'category']
for col in cat_cols:
    X[col] = X[col].astype('category')

In [ ]:
def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 5.0),
        'verbosity': -1,
        'random_state': 42
    }

    # K-Fold
    kf = KFold(n_splits=5, shuffle=True, random_state=810)
    auc_scores = []

    for train_idx, val_idx in kf.split(X_train, y_train):
        X_train_kf, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_kf, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        train_data = lightgbm.Dataset(X_train_kf, label=y_train_kf, categorical_feature=cat_cols)
        val_data = lightgbm.Dataset(X_val, label=y_val, reference=train_data)

        model = lightgbm.train(
            params,
            train_data,
            valid_sets=[val_data],
            num_boost_round=1000,
            callbacks=[
                lightgbm.early_stopping(stopping_rounds=50)
            ]
        )

        preds = model.predict(X_val)
        auc = roc_auc_score(y_val, preds)
        auc_scores.append(auc)
    # return np.mean(auc_scores)

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2025-07-20 16:28:12,799] A new study created in memory with name: no-name-332148bb-d7d1-4fe8-9a53-6b7f6cf0d2a6


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[223]	valid_0's auc: 0.775393
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[191]	valid_0's auc: 0.771365
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[229]	valid_0's auc: 0.770696
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[192]	valid_0's auc: 0.771969
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[168]	valid_0's auc: 0.772266


[I 2025-07-20 16:28:49,135] Trial 0 finished with value: 0.7723376943325464 and parameters: {'learning_rate': 0.21856769486173616, 'num_leaves': 93, 'max_depth': 4, 'min_child_samples': 22, 'subsample': 0.94185946168556, 'colsample_bytree': 0.7338199585439624, 'reg_alpha': 4.235584059867399, 'reg_lambda': 4.11120275038676}. Best is trial 0 with value: 0.7723376943325464.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[609]	valid_0's auc: 0.777426
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[558]	valid_0's auc: 0.774001
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[555]	valid_0's auc: 0.773583
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[456]	valid_0's auc: 0.773273
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[598]	valid_0's auc: 0.77457


[I 2025-07-20 16:30:32,701] Trial 1 finished with value: 0.7745704854258376 and parameters: {'learning_rate': 0.0918395326341712, 'num_leaves': 63, 'max_depth': 4, 'min_child_samples': 12, 'subsample': 0.5375048764847135, 'colsample_bytree': 0.5732817579410701, 'reg_alpha': 0.1531760957323558, 'reg_lambda': 4.4341877720074185}. Best is trial 1 with value: 0.7745704854258376.


...

[I 2025-07-20 18:10:10,117] Trial 47 finished with value: 0.773811019910488 and parameters: {'learning_rate': 0.06396003599543293, 'num_leaves': 68, 'max_depth': 10, 'min_child_samples': 59, 'subsample': 0.9165502260238114, 'colsample_bytree': 0.7323311997055609, 'reg_alpha': 1.2646656335830155, 'reg_lambda': 3.015833122916267}. Best is trial 27 with value: 0.7755319343232606.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[356]	valid_0's auc: 0.775332
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[350]	valid_0's auc: 0.773726
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[360]	valid_0's auc: 0.773407
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[356]	valid_0's auc: 0.773787
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[434]	valid_0's auc: 0.774094


[I 2025-07-20 18:12:58,863] Trial 48 finished with value: 0.7740692818446819 and parameters: {'learning_rate': 0.04519170467620538, 'num_leaves': 77, 'max_depth': 9, 'min_child_samples': 51, 'subsample': 0.9696747806735496, 'colsample_bytree': 0.7539301407795017, 'reg_alpha': 2.0054050205992477, 'reg_lambda': 2.062512322485692}. Best is trial 27 with value: 0.7755319343232606.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[108]	valid_0's auc: 0.77263
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[139]	valid_0's auc: 0.772572
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[85]	valid_0's auc: 0.767593
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[105]	valid_0's auc: 0.76996
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[96]	valid_0's auc: 0.771379


[I 2025-07-20 18:13:47,549] Trial 49 finished with value: 0.770826936683395 and parameters: {'learning_rate': 0.17639520311699755, 'num_leaves': 30, 'max_depth': 8, 'min_child_samples': 82, 'subsample': 0.7892272357267471, 'colsample_bytree': 0.6903755202836902, 'reg_alpha': 3.527094764409456, 'reg_lambda': 2.351588561318593}. Best is trial 27 with value: 0.7755319343232606.


In [51]:
print(f'Best AUC: {study.best_value:.4f}')
print(study.best_params)
config_path = '../configs/model_config.yaml'
with open(config_path, 'w') as file:
    yaml.dump(study.best_params, file, default_flow_style=False)
print('Dumped params')

Best AUC: 0.7755
{'learning_rate': 0.015623696975699715, 'num_leaves': 51, 'max_depth': 10, 'min_child_samples': 72, 'subsample': 0.9195509857832559, 'colsample_bytree': 0.547460031724486, 'reg_alpha': 2.2628683412386446, 'reg_lambda': 2.2087198162054733}
Dumped params
